In [187]:
import z3
import pandas as pd
import pprint
import timeit
import numpy as np
import random
import json

import networkx as nx

#----------------------------------------------------------------
#   Functions
#----------------------------------------------------------------
def a2nVars(appList, nodeList):
    '''Builds an m x n pandas datafram with apps and rows(index) and nodes as columns'''
    a2n = [ [ z3.Real("a%s%s" % (i+1, j+1)) 
         for j in range(len(nodeList)) ]
       for i in range(len(appList))]
    
    a2n_df = pd.DataFrame(a2n, index=appList, columns=nodeList)
    
    return a2n_df

def DependsOn(a2n_df, app, deps):
    #res = [z3.Implies(a2n_df.loc[app].sum()>0, a2n_df.loc[dep].sum()>0) for dep in deps]
    #print("res %s" %res)
    return(z3.And([z3.Implies(a2n_df.loc[app].sum()>0, a2n_df.loc[dep].sum()>0) for dep in deps]))

def setA2NDomain(a2n_df):
    a2n_domain = [z3.Or(a2n_df.loc[app, node]==0, a2n_df.loc[app, node]==1) \
                  for app in a2n_df.index \
                  for node in a2n_df.columns]
    return a2n_domain

def setRsrcConstraint(a2n_df, rpa_df, rpn_df):
    app_Load = rpa_df.dot(a2n_df)
    #display(app_Load)
    node_Surplus = rpn_df - app_Load
    #display(node_Surplus)
    
    rsrc_constraint = [node_Surplus.loc[rsrc,node]>=0 \
                       for rsrc in rpa_df.index \
                       for node in a2n_df.columns]
    return rsrc_constraint

def setMDDConstraint(a2a_df, a2n_df):
    mdd = list()
    for app in a2n_df.index:
        if a2a_df.loc[app].sum() > 0: #only put in the constraint if the app has dependencies
            deps = [dep for dep in a2a_df.loc[app].index if a2a_df.loc[app,dep]==1]
            #print("deps %s" %deps)
            mdd.append(DependsOn(a2n_df, app, deps))
    #print("MDD %s" %mdd)
    return mdd

#if an app was placed in initial deployment, it must be placed at least once in final deployment.  
def setMPIConstraint(placeable, a2n_df): 
    MPI = list()
    for app in placeable.index:
        MPI.append(placeable.loc[app].sum()>0)
    return MPI

def setConstraints(a2a_df, a2n_df, rpa_df, rpn_df, placeable):
    '''adds constraints to the values in an matrix'''
    a2n_domain = setA2NDomain(a2n_df)       
    rsrc_constraint = setRsrcConstraint(a2n_df, rpa_df, rpn_df)    
    mdd = setMDDConstraint(a2a_df, a2n_df)    
    mpi = setMPIConstraint(placeable, a2n_df)
    return (a2n_domain + rsrc_constraint + mdd + mpi)

def getNodeUtil(a2n_df, rpn_df):
    node_load = rpa_df.dot(a2n_df)
    node_util = node_load.divide(rpn_df) #pandas dataframe
    z3simplifyDF = lambda x: z3.simplify(x)    
    return node_util.applymap(z3simplifyDF)

def getNetworkUtil(a2n_df, rpa_df, rsrc):
    '''Returns network utilization. Can potentially be used for balancing later. All pieces are available'''
    # sum 1 to m a_ij*c
    node_load = rpa_df.dot(a2n_df) #pandas dataFrame dot operator
    
    #R(a)
    def getNetworkLoad(node_load, rsrc):
        '''Returns a z3.ArithRef of a sum of all apps on all nodes'''
        network_load = (node_load.loc[rsrc].values).sum()
        return network_load
    
    network_load = getNetworkLoad(node_load, rsrc)    
    network_supply = rpn_df.loc[rsrc].sum()
    node_util = node_load.divide(rpn_df)
    network_util = network_load/(network_supply)
    return network_util

if False:
    def latencyCost(a2a_df, a2n_df, n2n_df):
        numNodes = len(a2n_df.columns) # number of FSSNs
        markings = list() # The equations for each possible placement of the dependencies
        #list of all apps that have dependencies
        hasDeps = [node for node in a2n_df.index if a2a_df.loc[node].sum()>0]
        #print(hasDeps)
        #n nodes for primary app
        #2^(n)-1 placements for a particular dependency
        #should be 2^(n)-1 x n x sum of all dependencies
        #eg. n=5 app1 has 2 deps app2 has 1 dep
        # count = 5 x 31 x 3 = 465
        for srcNode in a2n_df.columns:
            #print("new sources")
            for pr_app in hasDeps:
                #print("pr_app %s" %pr_app)
                #print("check if %s is true: %s" %(a2n_df.loc[pr_app, srcNode]==1, bool(a2n_df.loc[pr_app, srcNode]==1)))
                if a2n_df.loc[pr_app, srcNode] ==1: #app that has deps is on srcNode
                    ## ANOTHER FOR LOOP IF THERE ARE MULTIPLE DEPENDENCIES
                    primaryDeps = [dep for dep in a2n_df.index if a2a_df.loc[pr_app, dep]==1]
                    for dep in primaryDeps:
                        #print("new dep")
                        # iterate through the 2^numNodes potential dependency placements
                        # and save them as binary representation to look up when running the optimizer
                        for i in range(2**numNodes):
                            mark = format(i, 'b').zfill(numNodes) # express placment in binary    
                            #print(mark)
                            # The dependency must be place somewhere. So this skips the case where i=0
                            # This may not be necessary thanks to the constraint. 
                            if mark == format(0,'b').zfill(numNodes):
                                continue
                            marking = list() # list for placment equations e.g (1 - a11)*(1 - a12)*a13
                            latency = [] # latencies between node hosting primary app and all nodes hosting the dependency for a particular placement. 
                            #print ("-----NEW MARKING-----")
                            # iterate through nodes using the binary placement to generate marking equation
                            for i, node in enumerate(a2n_df.columns):
                                #print ("mark %s" %mark)
                                #print ("i %s " %i)
                                #print ("mark[i] %s" %type(int(mark[i]))) 
                                # add marking to list and if binary has a 1, add latency to list of possible latencies
                                if int(mark[i]) == 1:
                                    marking.append(a2n_df.loc[dep, node]) #a_ij
                                    latency.append(n2n_df.loc[srcNode, node])
                                    #latency.append(random.randint(0,10))
                                if int(mark[i]) == 0:
                                    marking.append(1 - a2n_df.loc[dep,node]) # 1- a_ij
                            #print("latency %s" %latency)                    
                            #print("min latency %s" %min(latency).item())
                            #print("min latency %s" %min(latency))
                            #print("type(min(latency)) = %s" %type(min(latency)))
                            #print("marking: %s" %marking)
                            #print("marking * latency: %s" %(z3.Product(marking) * min(latency).item()))
                            #print("marking * latency: %s" %(z3.Product(marking) * min(latency)))
                            #markings.append(z3.Product(marking) * min(latency).item())
                            markings.append(z3.Product(marking) * min(latency))
        return markings

def scaleLatency(a2a_df, numNodes, max_edge_latency):
        min_latency = 0 #app is deployed everywhere
        #max_latency = max([sum(self.n2n_df.loc[node, :]) for node in self.nodeList]) #app is deployed on node with highest total latency
        #max_deps = max([(a2a_df.loc[app]).sum() for app in a2a_df.columns])
        # SCALING LATENCY THIS WAY CAUSES COST TO STAY THE SAME EVEN IF THE NUMBER OF DEPENDENCIES CHANGE. NEED 
        # TO CHECK IF I CAN GET A scale*latency = 1
        max_deps = sum(a2a_df.values.flatten())
        max_latency = max_edge_latency * (numNodes-1) * max_deps
        scale_latency = 1/(max_latency - min_latency)
        return scale_latency

def a2nEval(a2n_df, model):
    a2n_df_r = a2n_df.copy()
    #deps = [dep for pr in a2n_df.index for dep in a2n_df.index if a2a_df.loc[pr, dep]==1]
    for app in a2n_df_r.index:
        for node in a2n_df.columns:
            if(type (a2n_df_r.loc[app,node]) == z3.ArithRef):
                a2n_df_r.loc[app, node] = model.eval(a2n_df_r.loc[app, node])
    return a2n_df_r
    

def getF(a2a_df, a2n_df, n2n_df, rpa_df, p, q, sym):
    #display(a2n_df_r)
    network_util_r = z3.simplify(getNetworkUtil(a2n_df, rpa_df, rpa_df.index[-1]))
    #latency_r = z3.Sum(latencyCost(a2a_df, a2n_df, n2n_df))
    latency_r = LatencyCost(a2a_df, a2n_df_raw, a2n_df, n2n_df, placeable)
    numNodes = len(a2n_df.columns)
    max_edge_latency = max(n2n_df.values.flatten())
    scale_latency = scaleLatency(a2a_df, numNodes, max_edge_latency=max_edge_latency) #numpy.float64
    f = p*network_util_r + (1-p)*latency_r*scale_latency
    
    if sym == False:
        f_r = float(z3.simplify(f).as_decimal(512)[0:-1])
        print("f = p*network_util_r + (1-p)*latency_r*scale_latency")
        print( "f = " + str(p) + " * " + network_util_r.as_decimal(10) + " + " + 
                        str("%.2f" %(1.0-p))+ " * " + z3.simplify(latency_r).as_decimal(5) + "*" + 
                        str(scale_latency))
        print("f = " +  z3.simplify(p*network_util_r).as_decimal(5) + "  +  " + 
                        z3.simplify((1-p)*latency_r*scale_latency).as_decimal(10))
    else:
        f_r = z3.simplify(f)
    return f_r

def z3simplify(x):
    #print(type(x))
    if type(x) == z3.ArithRef:
        newx = z3.simplify(x)
    else:
        newx = x
    return newx

def LatencyObj_df(a2a_df, a2n_df_raw, a2n_df, n2n_df, placeable):
    def getbMarks(numNodes):
        bmarks = list()
        for i in range(2**numNodes):
            bmark = format(i, 'b').zfill(numNodes) # express placment in binary    
            bmarks.append(bmark)
        return bmarks
            
    def getAppDeps(app):
        return [dep for dep in a2a_df.columns if a2a_df.loc[app, dep]==1]
        
    numNodes = len(n2n_df.columns)
    hasDeps = [app for app in a2a_df.index if a2a_df.loc[app].sum()>0]                 
    bmarks = getbMarks(len(n2n_df.columns))[1:-1]
    
    objective_df = pd.DataFrame()
    for papp in hasDeps:        
        for srcNode in n2n_df.index:                
            p_aij = a2n_df.loc[papp, srcNode]
            p_aij_raw = a2n_df_raw.loc[papp, srcNode]
            #print("papp on node: %s" %p_aij_raw)
            deps = getAppDeps(papp)
            for dep in deps:   
                #print("new dependency")
                objs = list()
                for bmark in bmarks:
                    aijs = a2n_df.loc[dep]
                    marking = list()
                    latency = list()
                    for ix, bm in enumerate(bmark):
                        aij = aijs[ix]
                        dstNode = n2n_df.index[ix]
                        if int(bm) == 1:
                            marking.append(aij)
                            latency.append(n2n_df.loc[srcNode, dstNode])
                        if int(bm) ==0:
                            marking.append(1-aij)
                    #print(" %s * %s * min%s" %(p_aij_raw, marking, latency))
                    obj = z3.Product([p_aij] + marking + [min(latency)])
                    objs.append(obj)                
                objective_df.set_value(str(p_aij_raw), dep, z3.Sum(objs))
    return (objective_df)

def LatencyCost(a2a_df, a2n_df_raw, a2n_df, n2n_df, placeable):
    lat_obj_df=LatencyObj_df(a2a_df, a2n_df_raw, a2n_df, n2n_df, placeable)
    #display(lat_obj_df.loc['a11', 'BRM'])
    flat = sum(lat_obj_df.stack().groupby(level=0).apply(list).tolist(), [])
    lat_obj = z3.Sum(flat)
    return lat_obj

In [178]:
def iterSat(a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p, q, iters, placeable, ):
    sat_loops = 0
    max_loops = iters
    s = z3.Solver()   
    constraints = setConstraints(a2a_df, a2n_df, rpa_df, rpn_df, placeable)
    s.add(constraints)
    objF = z3.Real("objF")
    f = getF(a2a_df, a2n_df, n2n_df, rpa_df, p, q, sym=True)
    s.add(objF == f)
    
    start = timeit.default_timer()
    
    #f_old = -1
    while (s.check() == z3.sat and sat_loops<max_loops):
        m = s.model()
        #display(m)    
        if(sat_loops>0):
            s.pop()
        s.push()
        s.add(objF<m[objF])
        sat_loops = sat_loops+1       
    stop = timeit.default_timer()
    print ("sat_loops = %s" %sat_loops)
    print ("run time: %s" %(stop - start))
    try:
        f_r = m[objF].as_decimal(5)
        a2n_df_r = a2nEval(a2n_df, m)
        return(a2n_df_r, f_r)
    except Exception as inst:
        print(inst)
        print("No solution")
        

In [3]:
def optimize(a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p, q, placeable):
    opt = z3.Optimize()       
    constraints = setConstraints(a2a_df, a2n_df, rpa_df, rpn_df, placeable)
    opt.add(constraints)      
    objF = z3.Real("objF")
    f = getF(a2a_df, a2n_df, n2n_df, rpa_df, p, q, sym=True)  
    opt.add(objF == f)    
    
    start = timeit.default_timer()
    opt.minimize(objF)
    
    sat = opt.check()
    opt_model = opt.model()
    #print("optimization model \n %s \n" %opt_model)
    stop = timeit.default_timer()
    print ("run time: %s" %(stop - start))
    
    f_r = opt_model[objF].as_decimal(5) 
    a2n_df_r = a2nEval(a2n_df, opt_model)        
    return a2n_df_r, f_r, 
    

In [52]:
import operator
from collections import Counter

def bruteForce(placeable, a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p , q):    
    numVars = len(placeable.values.flatten())#a_ijs in numpy array from pandas dataframe excludes initial placement
    my_list = list()
    solutions = list()
    
    s = z3.Solver()       
    constraints = setConstraints(a2a_df, a2n_df, rpa_df, rpn_df, placeable)
    s.add(constraints)
    objF = z3.Real("objF")
    f = getF(a2a_df, a2n_df, n2n_df, rpa_df, p, q, sym=True)
    s.add(objF == f)    
    
    start = timeit.default_timer()
    min_found = "5"
    
    for i in range(2**numVars):
        marking = format(i, 'b').zfill(numVars) 
        m = 0
        s.push()
        for app in placeable.index:
            for node in placeable.columns:
                s.add(a2n_df.loc[app,node] == z3.RealSort().cast(marking[m]))
                m = m + 1  
        #check placement for feasibility
        issat = s.check()
        #print(issat)
        if (issat == z3.sat):
            last_model = s.model()
            f_r = last_model[objF].as_decimal(10)
            #print(f_r)
            if (f_r < min_found):
                #print("f_r:%s is smaller than %s" %(f_r, min_found))
                min_found = f_r                
            my_list.append(f_r)
            solutions.append(last_model)
            #if len(solutions)>10:
            #    break                
        s.pop()

    stop = timeit.default_timer()    
    print ("run time: %s" %(stop - start))
    
    max_index, max_value = max(enumerate(my_list), key=operator.itemgetter(1))
    min_index, min_value = min(enumerate(my_list), key=operator.itemgetter(1))

    min_solution = solutions[min_index]
    max_solution = solutions[max_index]
    #print(Counter(my_list))
    
    #print("numSolutions: %s" %len(solutions))
    
    f_r = min_solution[objF].as_decimal(10)
    num_found = my_list.count(f_r)
    #print("num_found %s" %num_found)
    a2n_df_r = a2nEval(a2n_df, min_solution)
    return(a2n_df_r, f_r, num_found)    

def save2Json(a2a_df, a2n_df, a2n_df_r, n2n_df, rpa_df, rpn_df):
    p = {
        "a2a" : a2a_df.to_json(orient='split'), 
        "a2n" : a2n_df.applymap(str).to_json(orient='split'),
        "a2n_r" : a2n_df_r.applymap(lambda x: int(str(x))).to_json(orient='split'),
        "n2n" : n2n_df.to_json(orient='split'),
        "rpa" : rpa_df.to_json(orient='split'),
        "rpn" : rpn_df.to_json(orient='split')       
    }
    with open('res.txt', 'a') as outfile:
        json.dump(save2Json(a2a_df, a2n_df, a2n_df_r, n2n_df, rpa_df, rpn_df), outfile)
    return p

def symDF(rows, columns, _min, _max):
    '''use to generate a2a and n2n matrices'''
    m = len(rows)
    n = len(columns)
    rng = np.random.randint(_min, _max, size=(m,n ))
    lower = np.tril_indices(m, -1) #indices for lower triagle below diagonal
    rngSymDF[lower]=rngSymDF.T[lower]
    np.fill_diagonal(rngSymDF, 0)
    rngSym_df = pd.DataFrame(rngSymDF, index=rows, columns=columns)
    return rngSymDF


In [195]:
def randomTest(m, n, arsrc, nrsrc, numRsrc, minL, maxL):
    appList = ["a%s"%(i+1) for i in range(m)]
    nodeList = ["n%s"%(i+1) for i in range(n)]
    resourceTypes = ["r%s"%(i+1) for i in range(numRsrc)]
    a2n_df = a2nVars(appList, nodeList)
    placeable = a2n_df.copy()
    p = round(random.random(), 1)
    q = 0
    rng_rpa = np.random.choice(arsrc, size=(numRsrc,m))
    rpa_df = pd.DataFrame(rng_rpa, index=resourceTypes, columns=appList)
    rng_rpn = np.random.choice(nrsrc, size=(numRsrc,n))
    rpn_df = pd.DataFrame(rng_rpn, index=resourceTypes, columns=nodeList)
    display(rpn_df)
    
    n2n_df = symDF(nodeList, nodeList, minL, maxL)
    #print(n2n_df)
    a2a = np.random.randint(0, 2, size=(m,n))
    np.fill_diagonal(a2a, 0)
    a2a_df = pd.DataFrame(a2a, index=appList, columns=appList)
    #print(a2a_df)
    display(rpn_df.loc[rpa_df.index[-1]])
    
    if True:
        print("-------------------------------------------------------")
        print("        ITERATE SAT")
        print("-------------------------------------------------------")
        a2n_df_r, f_r = iterSat(a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p, q, iters=100, placeable=placeable)   
        print("itersat found: %s " %f_r)
        display(a2n_df_r)
        getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)

        print("-------------------------------------------------------")
        print("        BRUTE FORCE")
        print("-------------------------------------------------------")
        a2n_df_r, f_r, num_found = bruteForce(placeable, a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p , q)
        print("brute min cost found: %s" %f_r)
        print("number of solutions with same cost: %s " %num_found)
        display(a2n_df_r)
        getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)
        print("-------------------------------------------------------")
        print("        OPTIMIZE")
        print("-------------------------------------------------------")
        a2n_df_r, f_r = optimize(a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p, q, placeable)
        print("Optimize found: %s " %f_r)
        display(a2n_df_r)
        getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)
        print("--------------------------------------------------------")

        display(LatencyObj_df(a2a_df, a2n_df_raw, a2n_df_r, n2n_df, placeable).applymap(z3simplify))
        display(getNodeUtil(a2n_df_r, rpn_df))
   
    
randomTest(4, 4, arsrc=[50, 100, 150, 200], nrsrc=[1024, 2048, 4096, 8192], numRsrc=1, minL=10, maxL=100)

,n1,n2,n3,n4
r1,2048,1024,4096,1024


n1    2048
n2    1024
n3    4096
n4    1024
Name: r1, dtype: int64

-------------------------------------------------------
        ITERATE SAT
-------------------------------------------------------


KeyError: 'the label [r1] is not in the [index]'

In [159]:
if True:
    #----------------------------------------------------------------
    #          GIVENS
    #----------------------------------------------------------------
    appList = sorted(['APP1', 'APP2', 'APP3'])
    nodeList = sorted(['FSSN3', 'FSSN2', 'FSSN1', 'FSSN4', 'FSSN5'])
    resourceTypes = ['storage', 'ram']#('cpu', 'ram', 'storage')
    # ----- APPS x Resources -------
    rpa = [[2048, 1024, 1024],
           [1024, 512, 512]]
    rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)

    rpn = [[ 8192,  8192, 16384,  8192,  8192],
           [ 4096,  4096,  4096,  4096,  4096]]
    rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)

    # Rows depend on columns
    a2a = [[0, 1, 1],
           [0, 0, 0],
           [0, 0, 0]]
    #a2a = [[0, 1, 1],
    #       [0, 0, 1],
    #       [0, 1, 0]]
    a2a_df = pd.DataFrame(a2a, index=appList, columns=appList) 

    n2n = [[0, 3, 4, 5, 6],
           [3, 0, 5, 6, 7],
           [4, 5, 0, 7, 8],
           [5, 6, 7, 0, 9],
           [6, 7, 8, 9, 0]]

    #n2n = [[0, 10, 10, 10, 10],
    #       [10, 0, 10, 10, 10],
    #       [10, 10, 0, 10, 10],
    #       [10, 10, 10, 0, 10],
    #       [10, 10, 10, 10, 0]]

    n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList) 

    #Using a function for now, but that won't be the case in the end. 
    def initial_deployment(a2n_df):
            a2n_df.loc['APP1'] = 1
            placeable = a2n_df.drop(['APP1'])
            return a2n_df, placeable

    #----------------------------------------------------------------
    #         Derived
    #----------------------------------------------------------------
    # make list of dependencies
    a2n_df_raw = a2nVars(appList, nodeList)
    #display(a2n_df)
    a2n_df, placeable = initial_deployment(a2n_df_raw.copy())

    p = .7
    q = 0

In [192]:
rpa_df.index[-1]

'ram'

In [194]:
rpn_df.loc[rpa_df.index[-1]]

FSSN1    4096
FSSN2    4096
FSSN3    4096
FSSN4    4096
FSSN5    4096
Name: ram, dtype: int64

In [6]:
#rpa_df.loc['ram', "APP2"] = 1024 #increase from 512. Reduce instances of app 2.  TRUE 2->1
#rpa_df.loc['ram', "APP2"] = 256 #decrease from 512. increase instances of app 2. TRUE 2->5
#rpa_df.loc['ram', "APP3"] = 1024 #increase from 512. Reduce instances of app 3.  TRUE 2->1
#rpa_df.loc['ram', "APP3"] = 256 #decrease from 512. increase instances of app 3.  TRUE 2->5

#rpn_df.loc['ram', 'FSSN2'] = 14096 #increase from 4096. Incrase # of apps on FSSN2.  ?
# Did not exclusively increase the number of apps on 2. Increased the total number of floating apps from 6->8
# This makes sense becasue the utilization term considers the system. There is no reason to put more on 2. I was 
# mistaken. 

#rpn_df.loc['ram', 'FSSN2'] = 2048 #decrease from 4096. Decrease # of apps on system. TRUE 6-> 4 (only solution)
#rpn_df.loc['ram', 'FSSN2'] = 1024 #decrease from 4096. Decrease # of apps on system. TRUE 6-> 4 (only solution)
#rpn_df.loc['ram', 'FSSN2'] = 1023 # no solution see todo)

#rpn_df.loc['ram', 'FSSN1'] = 1024 #decrease from 4096. Expect 1 app to move from FSSN1 to FSSN2. TRUE
# Had bug in resource constraint. Just had sum of apps must exceed 0. Not (Resource available - sum of apps > 0)

#n2n_df.loc['FSSN1', 'FSSN4'] = 1# Remove apps 2and3 from FSSN4 TRUE
#n2n_df.loc['FSSN4', 'FSSN1'] = 1#

#n2n_df.loc['FSSN1', 'FSSN3'] = 10# Add app 2 and 3 to FSSN3 True
#n2n_df.loc['FSSN3', 'FSSN1'] = 10#

#a2a_df.loc["APP3", "APP2"] = 1 #Prior cost is: Cost is .3325. Cause app2 to depend on app3. new Cost is .32249.
# If the same deployment is used with the new dep added the cost is still .3325. 
# If the new second deployment is used with prior deps cost is .34125

#p = .9 # resource utilization more important. Deploy fewer apps 6-> 2
#p = .6 # latency is more important. Deploy more apps 6->8
# p = .5 # latency is more important. Deploy more apps 6->10(max deployment)
#---------------------------------------------------------------- 
#        Use
#----------------------------------------------------------------
if True:    
    print("-------------------------------------------------------")
    print("        ITERATE SAT")
    print("-------------------------------------------------------")
    a2n_df_r, f_r = iterSat(a2a_df, a2n_df, n2n_df, rpa_df, p, q, iters=100, placeable=placeable)   
    print("itersat found: %s " %f_r)
    display(a2n_df_r)
    getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)

    print("-------------------------------------------------------")
    print("        BRUTE FORCE")
    print("-------------------------------------------------------")
    a2n_df_r, f_r, num_found = bruteForce(placeable, a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p , q)
    print("brute min cost found: %s" %f_r)
    print("number of solutions with same cost: %s " %num_found)
    display(a2n_df_r)
    getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)
    print("-------------------------------------------------------")
    print("        OPTIMIZE")
    print("-------------------------------------------------------")
    a2n_df_r, f_r = optimize(a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p, q, placeable)
    print("Optimize found: %s " %f_r)
    display(a2n_df_r)
    getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)
    print("--------------------------------------------------------")

    getNodeUtil(a2n_df_r, rpn_df)

-------------------------------------------------------
        ITERATE SAT
-------------------------------------------------------
sat_loops = 11
run time: 4.835932247000528
itersat found: 0.33833? 


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1


f = p*network_util_r + (1-p)*latency_r*scale_latency
f = 0.7 * 0.4 + 0.30 * 14*0.0138888888889
f = 0.28  +  0.0583333333?
-------------------------------------------------------
        BRUTE FORCE
-------------------------------------------------------
run time: 5.139682412000184
brute min cost found: 0.3383333333?
number of solutions with same cost: 1 


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1


f = p*network_util_r + (1-p)*latency_r*scale_latency
f = 0.7 * 0.4 + 0.30 * 14*0.0138888888889
f = 0.28  +  0.0583333333?
-------------------------------------------------------
        OPTIMIZE
-------------------------------------------------------
run time: 15.901342135999585
Optimize found: 0.33833? 


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1


f = p*network_util_r + (1-p)*latency_r*scale_latency
f = 0.7 * 0.4 + 0.30 * 14*0.0138888888889
f = 0.28  +  0.0583333333?
--------------------------------------------------------


In [166]:
getNodeUtil(a2n_df_r, rpn_df)

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
storage,1/2,1/4,1/8,1/2,1/2
ram,1/2,1/4,1/4,1/2,1/2


In [7]:
LatencyObj_df(a2a_df, a2n_df_raw, a2n_df_r, n2n_df, placeable).applymap(z3simplify)

,APP2,APP3
a11,0,0
a12,3,3
a13,4,4
a14,0,0
a15,0,0


In [8]:
a2n_df_r

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1


Sweep as set of latency values to see if the solution methods ever reach distinct solutions. 

In [51]:

symDF(nodeList,nodeList, 1, 100)

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
FSSN1,0,64,95,84,57
FSSN2,64,0,68,7,39
FSSN3,95,68,0,35,5
FSSN4,84,7,35,0,72
FSSN5,57,39,5,72,0


In [9]:
if False:    
    for i in range(10):
        #random symmetric matrix
        rn2n = np.random.randint(1,4, size=(len(nodeList), len(nodeList)))
        lower = np.tril_indices(len(nodeList), -1)
        rn2n[lower]=rn2n.T[lower]
        np.fill_diagonal(rn2n, 0)
        rn2n_df = pd.DataFrame(rn2n, index=nodeList, columns=nodeList)
        display(rn2n_df)


        a2n_iter, f_iter = iterSat(a2a_df, a2n_df, rn2n_df, rpa_df, p, q, iters=100, placeable=placeable)  
        a2n_brut, f_brut, num_found = bruteForce(placeable, a2a_df, a2n_df, rn2n_df, rpa_df, rpn_df, p , q)
        print("number of solutions with same cost: %s " %num_found)
        a2n_opti, f_opti = optimize(a2a_df, a2n_df, rn2n_df, rpa_df, rpn_df, p, q, placeable)

        print(f_iter)
        print(f_brut)
        print(f_opti)
        print(f_iter == f_brut and f_iter==f_opti)

        display(a2n_iter)

ToDo
Handle error when there is no solution

# Demos

## 3 node manual example

In [10]:
if False:
    #----------------------------------------------------------------
    #          GIVENS
    #----------------------------------------------------------------
    appList = sorted(['APP1', 'APP2', 'APP3'])
    nodeList = sorted(['FSSN1','FSSN2','FSSN3'])
    resourceTypes = ['ram']#('cpu', 'ram', 'storage')
    # ----- APPS x Resources -------
    rpa = [[3, 2, 1]]
    rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)

    rpn = [[3, 4, 5]]
    rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)

    # Rows depend on columns
    a2a = [[0, 1, 1],
           [0, 0, 0],
           [0, 0, 0]]
    a2a_df = pd.DataFrame(a2a, index=appList, columns=appList) 

    n2n = [[0, 1, 3],
           [1, 0, 2],
           [3, 2, 0]]
    n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList) 

    #Using a function for now, but that won't be the case in the end. 
    def initial_deployment(a2n_df): 
        a2n_df.loc['APP1'] = 0 #z3.Real('0')
        a2n_df.loc['APP1', 'FSSN2'] = 1# z3.Real('1')
        placeable = a2n_df.drop(['APP1'])
        return a2n_df, placeable

    #----------------------------------------------------------------
    #         Derived
    #----------------------------------------------------------------
    # make list of dependencies
    a2n_df_raw = a2nVars(appList, nodeList)
    #display(a2n_df)
    a2n_df, placeable = initial_deployment(a2n_df_raw.copy())

    p = .5#as long as p<1 ie latency matters, placment is the same. lower Cost with lower p
    q = 0    
    
if False:
    a2n_df_r, f_iter = iterSat(a2a_df, a2n_df, n2n_df, rpa_df, p, q, iters=100, placeable=placeable)   
    display(a2n_df_r)
    a2n_df_r, f_brut, num_found = bruteForce(placeable, a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p , q)
    print("itersat found: %s " %f_iter)
    print("brute min cost found: %s num found: %s" %(f_brut, num_found))
    print("cost: %s" %getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False))

    util_verbose_raw = rpa_df.dot(a2n_df).applymap(z3simplify).divide(rpn_df)
    display(util_verbose_raw)

    util_verbose = rpa_df.dot(a2n_df_r.multiply(a2n_df)).applymap(z3simplify).divide(rpn_df)
    util_verb = rpa_df.dot(a2n_df_r).applymap(z3simplify).divide(rpn_df)
    display(util_verbose)
    display(util_verb)

## CEC use case example

In [11]:
if False:
    #----------------------------------------------------------------
    #          GIVENS
    #----------------------------------------------------------------
    appList = sorted(['AAM', 'BEA+XMPP', 'BRM', 'GRM', 'JS', 'Store', 'TOV', 'VG1', 'VG3'])
    nodeList = sorted(['Berkeley Cal', 'Berkeley Hills', 'North Berkeley', 'West Berkeley'])
    resourceTypes = ['storage', 'ram']#('cpu', 'ram', 'storage')
    # ----- APPS x Resources -------
    rpa = [[100, 100, 100,100, 100,100,100,100,100], # storage
           [50, 200, 50, 50, 100, 60, 50, 30, 70]] # ram
            
    rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)

    rpn = [[ 8192,  8192, 16384,  8192],
           [ 4096,  4096,  4096,  2048]]
    rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)
    
    # Rows depend on columns
    #'AAM', 'BEA+XMPP', 'BRM', 'GRM', 'JS', 'Store', 'TOV', 'VG1', 'VG3 
    # Everything depends on JS, so I'm including it with the local apps, as part of the initial deployment, 
    # since I'm only handling non-local dependencies. 
    a2a = [[0, 0, 1, 1, 0, 0, 0, 0, 0], # 'AAM' 
           [0, 0, 0, 0, 0, 1, 0, 0, 0], # 'BEA+XMPP'
           [0, 0, 0, 0, 0, 0, 0, 0, 0], # 'BRM'
           [0, 0, 0, 0, 0, 1, 0, 0, 0], # 'GRM'
           [0, 0, 0, 0, 0, 0, 0, 0, 0], # 'JS'
           [0, 0, 0, 0, 0, 0, 0, 0, 0], # 'Store'
           [0, 0, 0, 1, 0, 0, 0, 0, 0], # 'TOV'
           [0, 0, 0, 0, 0, 0, 0, 0, 0], # 'VG1'
           [0, 0, 0, 0, 0, 1, 0, 0, 0]] # 'VG3'
    a2a_df = pd.DataFrame(a2a, index=appList, columns=appList) 

    n2n = [[0, 3, 4, 5],
           [3, 0, 5, 6],
           [4, 5, 0, 7],
           [5, 6, 7, 0]]

    n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList) 

    #Using a function for now, but that won't be the case in the end. 
    def initial_deployment(a2n_df):
            a2n_df.loc['VG1'] = 1
            a2n_df.loc['VG3'] = 1
            a2n_df.loc['JS'] = 1
            
            placeable = a2n_df.drop(['VG1', 'VG3', 'JS'])
            return a2n_df, placeable

    #----------------------------------------------------------------
    #         Derived
    #----------------------------------------------------------------
    # make list of dependencies
    a2n_df_raw = a2nVars(appList, nodeList)
    #display(a2n_df)
    a2n_df, placeable = initial_deployment(a2n_df_raw.copy())

    p = .9#as long as p<1 ie latency matters, placment is the same. lower Cost with lower p
    q = 0   
if False:    
    print("-------------------------------------------------------")
    print("        ITERATE SAT")
    print("-------------------------------------------------------")
    a2n_df_r, f_r = iterSat(a2a_df, a2n_df, n2n_df, rpa_df, p, q, iters=10, placeable=placeable)   
    print("itersat found: %s " %f_r)
    display(a2n_df_r)
    getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)
if False:
    print("-------------------------------------------------------")
    print("        OPTIMIZE")
    print("-------------------------------------------------------")
    a2n_df_r, f_r = optimize(a2a_df, a2n_df, n2n_df, rpa_df, rpn_df, p, q, placeable)
    print("Optimize found: %s " %f_r)
    display(a2n_df_r)
    getF(a2a_df, a2n_df_r, n2n_df, rpa_df, p, q, sym=False)
    print("--------------------------------------------------------")  

In [12]:
LatencyObj_df(a2a_df, a2n_df_raw, a2n_df_r, n2n_df, placeable).applymap(z3simplify)

,APP2,APP3
a11,0,0
a12,3,3
a13,4,4
a14,0,0
a15,0,0
